# Goal here is to correct the label before the finetuning of ESM2
***
## I. Generate the DF_info and the DF_embeddings 
## II. Tryout multiple algoritms : 
> A. Ensemble method : Random Forest <br>
> B. Ensemble method : XGboost (?) <br>
> C. NN : Feed-forward Neural Network (FNN) <br>
> D. NN : Recurrent Neural Networks ; LSTM <br>
> E. NN : Graph Neural Networks  <br>
***

### I.

In [ ]:
df_prophage_id["prophage"] = df_prophage_id["prophage"].apply(lambda x : x.split(".fasta")[0])
df_prophage_id.columns = ["prophage_id", "Phage"]

DF_penultimate = DF_penultimate.merge(df_prophage_id , on = "Phage")
DF_penultimate.to_csv(f"{path_work}/DF_Dpo.final.2705.tsv", sep = "\t" , header = True, index = False)

embeddings_anubis = pd.read_csv(f"{path_work}/anubis.esm2.embedding.csv", sep = "," , header = None)
# embeddings_anubis = embeddings_anubis.drop([1281] , axis = 1)
# embeddings_anubis.to_csv(f"{path_work}/anubis.esm2.embedding.csv", sep = "," , header = None , index = False)

embeddings_dpo_b1 = pd.read_csv(f"{path_work}/Dpo.0805.embeddings.ultimate.csv", sep = "," , header = None)

embeddings_2705 = pd.concat([embeddings_dpo_b1 , embeddings_anubis], axis = 0)
embeddings_2705.to_csv(f"{path_work}/Dpo.2705.embeddings.ultimate.csv", sep = "," , header = None , index = False)


In [1]:
import torch
from torch.utils.data import Dataset
import os 
import pandas as pd
import warnings


/home/concha-eloko/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
path_work = "/media/concha-eloko/Linux/PPT_clean"

DF_info = pd.read_csv(f"{path_work}/DF_Dpo.final.2705.tsv", sep = "\t" ,  header = 0 )
DF_embeddings = pd.read_csv(f"{path_work}/Dpo.2705.embeddings.ultimate.csv", sep = ",", header= None )


In [33]:
DF_embeddings

,0,1,2,3,4,5,6,7,8,9,...,1271,1272,1273,1274,1275,1276,1277,1278,1279,1280
0,ppt__2930,-0.000061,-0.017329,0.012884,0.037123,-0.123747,0.004186,-0.061367,-0.056718,-0.037215,...,0.098806,0.012989,-0.001155,0.139749,-0.030987,0.059306,0.107041,-0.041463,-0.085581,0.114973
1,ppt__3300,0.004044,0.040011,-0.001234,-0.095745,-0.058056,-0.002394,0.007648,-0.059740,0.060850,...,-0.020369,0.016287,0.062586,-0.024336,0.019276,0.069623,0.035261,-0.118962,0.035672,0.085582
2,ppt__1182,0.018767,0.068116,-0.009109,-0.012598,-0.107001,0.011569,-0.030943,-0.045359,0.048923,...,0.014524,-0.024645,0.071878,0.018206,0.042790,0.088410,0.031970,-0.124592,0.070040,0.065348
3,ppt__3540,-0.028261,-0.047253,-0.027340,-0.052824,-0.089644,-0.023079,0.094861,0.026104,0.024001,...,0.051728,0.005634,-0.077874,0.030336,-0.037648,0.050625,0.046142,-0.158841,-0.007670,0.034556
4,ppt__942,0.014863,0.028030,0.014927,-0.025997,-0.096138,0.016290,0.015008,-0.066254,0.077959,...,0.008521,-0.019820,0.123201,-0.040306,0.030893,0.051362,0.047316,-0.102698,0.044830,0.084530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3603,anubis__304,0.006264,0.006471,-0.031665,0.078502,-0.131247,0.077167,0.043005,-0.183636,-0.022181,...,0.044299,-0.061847,0.017696,0.054798,-0.035830,-0.030202,0.039051,-0.127020,-0.113630,0.211258
3604,anubis__1273,-0.019114,0.063302,0.006635,-0.060343,-0.034054,-0.003895,0.033920,-0.080352,0.073579,...,-0.004504,-0.007906,0.075141,-0.052423,0.027127,0.073984,0.030664,-0.096409,0.011906,0.124885
3605,anubis__1311,0.051261,0.067942,0.005061,-0.019131,-0.060296,0.000984,0.037515,-0.033887,0.091774,...,0.044678,0.052609,0.112994,-0.000592,0.027122,0.086020,0.013660,-0.055491,0.021665,0.049301
3606,anubis__1525,-0.010655,0.083864,0.009084,-0.042220,-0.066479,0.008724,0.010109,-0.078033,0.065285,...,0.020752,0.024543,0.071302,0.035980,0.012171,0.054399,0.032167,-0.151018,0.042541,0.035221


> DataLoader 

In [ ]:
import os
import pandas as pd

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

***
### II. 

> LSTM

In [ ]:
import torch
from torch import nn

class SimpleModel(nn.Module):
    def __init__(self, prophage_vocab_size, prophage_embedding_dim, ancestor_vocab_size, ancestor_embedding_dim, depolymerase_embedding_dim, lstm_hidden_dim, lstm_layers, output_dim):
        super(SimpleModel, self).__init__()
        # Embedding layers for the categorical inputs
        self.prophage_embedding = nn.Embedding(prophage_vocab_size, prophage_embedding_dim)
        self.ancestor_embedding = nn.Embedding(ancestor_vocab_size, ancestor_embedding_dim)
        
        # LSTM for the depolymerase embeddings
        self.lstm = nn.LSTM(depolymerase_embedding_dim, lstm_hidden_dim, num_layers=lstm_layers, batch_first=True)

        # Fully connected layer for output
        self.fc = nn.Linear(prophage_embedding_dim + ancestor_embedding_dim + lstm_hidden_dim, output_dim)

    def forward(self, prophage, ancestor, depolymerase):
        prophage_embed = self.prophage_embedding(prophage)
        ancestor_embed = self.ancestor_embedding(ancestor)

        # The LSTM expects inputs of the form (batch_size, seq_length, num_features)
        lstm_out, _ = self.lstm(depolymerase.unsqueeze(0))

        # Concatenate the embeddings
        x = torch.cat((prophage_embed, ancestor_embed, lstm_out[0, -1]), dim=-1)

        return self.fc(x)

Optimization step

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

# Define the hyperparameter search space
search_space = {
    'prophage_embedding_dim': Integer(10, 300),
    'ancestor_embedding_dim': Integer(10, 300),
    'depolymerase_embedding_dim': Integer(10, 300),
    'lstm_hidden_dim': Integer(10, 300),
    'lstm_layers': Integer(1, 3),
    'output_dim': Integer(1, 10),
    'learning_rate': Real(1e-6, 1e-2, prior='log-uniform'),
}

# Wrap your PyTorch model in a scikit-learn compatible estimator
estimator = PyTorchEstimator(
    build_fn=lambda: SimpleModel(prophage_vocab_size=PROPHAGE_VOCAB_SIZE, ancestor_vocab_size=ANCESTOR_VOCAB_SIZE),
    criterion=nn.CrossEntropyLoss,
    optimizer=lambda params: torch.optim.Adam(params, lr=0.001),
    train_split=None,
    verbose=1,
)

# Initialize the BayesSearchCV object
bayes_search = BayesSearchCV(estimator, search_space, n_iter=50, cv=3)

# Perform the search
bayes_search.fit([prophage_data, ancestor_data, depolymerase_data], labels)


In [ ]:
prophage_data = torch.tensor(prophage_data)  # Shape: [num_samples]
ancestor_data = torch.tensor(ancestor_data)  # Shape: [num_samples]
depolymerase_data = torch.tensor(depolymerase_data)  # Shape: [num_samples, seq_length, num_features]
labels = torch.tensor(labels)  # Shape: [num_samples]

# Create the model
model = SimpleModel(prophage_vocab_size=PROPHAGE_VOCAB_SIZE, prophage_embedding_dim=PROPHAGE_EMBEDDING_DIM, ancestor_vocab_size=ANCESTOR_VOCAB_SIZE, ancestor_embedding_dim=ANCESTOR_EMBEDDING_DIM, depolymerase_embedding_dim=DEPOLYMERASE_EMBEDDING_DIM, lstm_hidden_dim=LSTM_HIDDEN_DIM, lstm_layers=LSTM_LAYERS, output_dim=OUTPUT_DIM)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(NUM_EPOCHS):
    model.train()
    
    # Forward pass
    outputs = model(prophage_data, ancestor_data, depolymerase_data)
    loss = criterion(outputs, labels)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, NUM_EPOCHS, loss.item()))


In [ ]:
def evaluate_model(model, depolymerase_test_data, labels_test_data, criterion):
    model.eval()  # switch the model to evaluation mode
    with torch.no_grad():  # don't compute gradients while evaluating
        outputs_test = model(depolymerase_test_data)
        loss_test = criterion(outputs_test, labels_test_data)
    print('Test Loss: {:.4f}'.format(loss_test.item()))

In [ ]:
# After initial training
evaluate_model(model, depolymerase_test_data, labels_test_data, criterion)


The transfer learning part

In [ ]:
class TransferLearningModel(nn.Module):
    def __init__(self, pretrained_model, new_input_dim, new_output_dim):
        super(TransferLearningModel, self).__init__()
        
        # Reuse the LSTM and fully connected layer from the pretrained model
        self.lstm = pretrained_model.lstm
        self.fc = nn.Linear(new_input_dim, new_output_dim)

    def forward(self, depolymerase):
        # Pass the data through the LSTM
        lstm_out, _ = self.lstm(depolymerase.unsqueeze(0))

        # Pass the output through the fully connected layer
        return self.fc(lstm_out[0, -1])

In [ ]:
evaluate_model(transfer_learning_model, depolymerase_test_data, labels_test_data, criterion)

***
> Graph Convolutional Network

https://www.youtube.com/watch?v=-UjytpbqX4A <br>
https://pytorch-geometric.readthedocs.io/en/latest/notes/heterogeneous.html<br>
https://www.youtube.com/watch?v=-UjytpbqX4A<br>
Inputs Explanation:

    data.x: This should be a [num_nodes, num_node_features] matrix containing your node features. In your case, these are the amino acid sequence and embedding representation of the depolymerase sequences.
    
    data.edge_index: This is your graph adjacency matrix, represented as a [2, num_edges] matrix where each column represents the two nodes that define an edge. The entries in edge_index are the indices of the nodes in the node feature matrix data.x.
    
    data.y: This should be a [num_nodes] vector containing your target labels. In your case, these are the true KL_type of each node.
    
    data.train_mask: This should be a [num_nodes] boolean vector indicating which nodes should be used for training. You could create training, validation, and test masks depending on how you want to split your data.

In [ ]:
import torch
from torch_geometric.nn import GCNConv
from torch.nn import functional as F

# Define a Graph Convolutional Network
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return x  # This is now your embedding

# Define a Classifier
class Classifier(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        self.fc = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return F.log_softmax(self.fc(x), dim=1)

# Bayesian search
def objective(trial):
    # Suggest values of the hyperparameters using a trial object.
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    layers = trial.suggest_int('layers', 1, 3)

    # Define and train your model here
    gcn_model = GCN(input_dim=INPUT_DIM, hidden_dim=HIDDEN_DIM, layers=layers)
    classifier_model = Classifier(input_dim=HIDDEN_DIM, output_dim=OUTPUT_DIM)

    gcn_optimizer = torch.optim.Adam(gcn_model.parameters(), lr=lr)
    classifier_optimizer = torch.optim.Adam(classifier_model.parameters(), lr=lr)

    # The rest of your training code here...
    # Be sure to return a value to minimize (e.g., validation loss)

    return val_loss

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

print('Best trial:')
trial = study.best_trial
print(' Value: ', trial.value)
print(' Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

# Instantiate the models
gcn_model = GCN(input_dim=INPUT_DIM, hidden_dim=HIDDEN_DIM)
classifier_model = Classifier(input_dim=HIDDEN_DIM, output_dim=OUTPUT_DIM)

# Define your optimizer and loss function for each model
gcn_optimizer = torch.optim.Adam(gcn_model.parameters(), lr=0.01)
classifier_optimizer = torch.optim.Adam(classifier_model.parameters(), lr=0.01)

loss_func = torch.nn.CrossEntropyLoss()

# Training loop for GCN
for epoch in range(200): # adjust number of epochs as needed
    gcn_model.train()
    gcn_optimizer.zero_grad()
    embeddings = gcn_model(data)  # forward pass

    # We don't have a loss for the GCN itself, because it's unsupervised. We just need the embeddings

# Training loop for Classifier
for epoch in range(200): # adjust number of epochs as needed
    classifier_model.train()
    classifier_optimizer.zero_grad()

    out = classifier_model(embeddings)  # forward pass using the embeddings as input

    # Compute the loss only for training nodes
    loss = loss_func(out[data.train_mask], data.y[data.train_mask]) 

    # Backward pass
    loss.backward()
    classifier_optimizer.step()

    # Print loss
    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, Loss: {loss.item()}")


> With heterogenous nodes

In [ ]:
from torch_geometric.data import Data

# Create feature matrices for prophages and bacteria nodes
prophage_features = torch.randn((num_prophages, num_prophage_features))  # Including depolymerase embeddings
bacteria_features = torch.randn((num_bacteria, num_bacteria_features))  # Including KL_type

# Create edge index for infection events
edge_index = torch.tensor([
    prophage_indices,  # Source nodes (prophages)
    bacteria_indices,  # Target nodes (bacteria)
], dtype=torch.long)

# Create data object
data = Data(
    x={'prophage': prophage_features, 'bacteria': bacteria_features},
    edge_index={'infection': edge_index},
    # Add any additional data here
)

In [ ]:
class HeteroGNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(HeteroGNN, self).__init__()
        self.conv_prophage = GCNConv(input_dim['prophage'], hidden_dim)
        self.conv_bacteria = GCNConv(input_dim['bacteria'], hidden_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x['prophage'] = self.conv_prophage(x['prophage'], edge_index['infection'])
        x['bacteria'] = self.conv_bacteria(x['bacteria'], edge_index['infection'])

        return x  # This is now your embedding


In [ ]:
import torch
from torch_geometric.nn import GCNConv
from torch.nn import functional as F

class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

# Instantiate the model with input, hidden and output dimensions
model = GCN(input_dim=INPUT_DIM, hidden_dim=HIDDEN_DIM, output_dim=OUTPUT_DIM)

# Define your optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_func = torch.nn.CrossEntropyLoss()

# Training loop
for epoch in range(200): # adjust number of epochs as needed
    model.train()
    optimizer.zero_grad()
    out = model(data) # forward pass

    # Compute the loss only for training nodes
    loss = loss_func(out[data.train_mask], data.y[data.train_mask]) 

    # Backward pass
    loss.backward()
    optimizer.step()

    # Print loss
    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, Loss: {loss.item()}")

In [ ]:
class GCNWithEmbeddings(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCNWithEmbeddings, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)  # The output is now an embedding
        self.classifier = torch.nn.Linear(hidden_dim, output_dim)  # This is a separate layer for classification

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)  # This is now your embedding

        out = self.classifier(x)  # This is your KL_type prediction

        return out, x  # Return both the prediction and the embedding
